In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2

In [3]:
import numpy as np

In [4]:
#Load YOLO
net = cv2.dnn.readNet("yolov3_custom3_final.weights","yolov3_custom3.cfg")
classes = []
with open("obj.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [5]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [6]:
colors= np.random.uniform(0,255,size=(len(classes),3))

In [7]:
#loading image
img = cv2.imread("test/IMG_1165.JPG")
img = cv2.resize(img,None,fx=0.4,fy=0.3)
height,width,channels = img.shape

In [8]:
blob = cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),True,crop=False)

In [14]:
#     for n,img_blob in enumerate(b):
#         cv2.imshow(str(n),img_blob)
        
net.setInput(blob)
outs = net.forward(outputlayers)
#print(outs[1])


#Showing info on screen/ get confidence score of algorithm in detecting an object in blob
class_ids=[]
confidences=[]
boxes=[]
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            #onject detected
            center_x= int(detection[0]*width)
            center_y= int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
        
            #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
            #rectangle co-ordinaters
            x=int(center_x - w/2)
            y=int(center_y - h/2)
            #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            boxes.append([x,y,w,h]) #put all rectangle areas
            confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
            class_ids.append(class_id) #name of the object tha was detected
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.5)

font = cv2.FONT_HERSHEY_PLAIN
lst = []
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h = boxes[i]
        yolo = x,y,w,h
        label = str(classes[class_ids[i]])
        #color = colors[i]
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(img,label,(x,y+30),font,1,(255,255,255),2)
        print(label)
        lst.append([x, y, label])
        print(x,y)
cv2.imshow("prediction",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

7
612 502
0
489 49
4
491 99
5
789 105
5
853 106
5
915 106
6
975 110
2
637 173
9
637 236
11
915 245
3
774 311
14
645 379
8
779 441
7
499 577
3
464 648
1
459 720


In [10]:
sorted(lst, key=lambda x: x[-2])

[[489, 49, '0'],
 [491, 99, '4'],
 [789, 105, '5'],
 [853, 106, '5'],
 [915, 106, '5'],
 [975, 110, '6'],
 [637, 173, '2'],
 [637, 236, '9'],
 [915, 245, '11'],
 [774, 311, '3'],
 [645, 379, '14'],
 [779, 441, '8'],
 [612, 502, '7'],
 [499, 577, '7'],
 [464, 648, '3'],
 [459, 720, '1']]

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(sorted(lst, key=lambda x: x[1]), columns=['X', 'Y', 'label'])
df.head()

,X,Y,label
0,489,49,0
1,491,99,4
2,789,105,5
3,853,106,5
4,915,106,5


In [13]:
df['label']

0      0
1      4
2      5
3      5
4      5
5      6
6      2
7      9
8     11
9      3
10    14
11     8
12     7
13     7
14     3
15     1
Name: label, dtype: object

In [35]:
df['label'].to_csv('test.txt', index=False, header=False)